In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import glob
import os
import sys
from collections import deque
from copy import deepcopy

import numpy as np
import torch
import torch.nn as nn
import yaml
import yamlordereddictloader

# from torchvision import models
from yaml import dump
from dlrm.dlrm_s_pytorch import DLRM_Net, dash_separated_ints, dash_separated_floats
from ir.handlers import handlers
from ir.trace import trace
from ir.trace import get_backprop_memory
from utils.logger import create_logger

In [3]:
# import sys
# import argparse
# import dlrm.dlrm_data_pytorch as dp
# ### parse arguments ###
# parser = argparse.ArgumentParser(
#     description="Train Deep Learning Recommendation Model (DLRM)"
# )
# # model related parameters
# parser.add_argument("--arch-sparse-feature-size", type=int, default=2)
# parser.add_argument(
#     "--arch-embedding-size", type=dash_separated_ints, default="4-3-2"
# )
# # j will be replaced with the table number
# parser.add_argument("--arch-mlp-bot", type=dash_separated_ints, default="4-3-2")
# parser.add_argument("--arch-mlp-top", type=dash_separated_ints, default="4-2-1")
# parser.add_argument(
#     "--arch-interaction-op", type=str, choices=["dot", "cat"], default="dot"
# )
# parser.add_argument("--arch-interaction-itself", action="store_true", default=False)
# # embedding table options
# parser.add_argument("--md-flag", action="store_true", default=False)
# parser.add_argument("--md-threshold", type=int, default=200)
# parser.add_argument("--md-temperature", type=float, default=0.3)
# parser.add_argument("--md-round-dims", action="store_true", default=False)
# parser.add_argument("--qr-flag", action="store_true", default=False)
# parser.add_argument("--qr-threshold", type=int, default=200)
# parser.add_argument("--qr-operation", type=str, default="mult")
# parser.add_argument("--qr-collisions", type=int, default=4)
# # activations and loss
# parser.add_argument("--activation-function", type=str, default="relu")
# parser.add_argument("--loss-function", type=str, default="mse")  # or bce or wbce
# parser.add_argument(
#     "--loss-weights", type=dash_separated_floats, default="1.0-1.0"
# )  # for wbce
# parser.add_argument("--loss-threshold", type=float, default=0.0)  # 1.0e-7
# parser.add_argument("--round-targets", type=bool, default=False)
# # data
# parser.add_argument("--data-size", type=int, default=1)
# parser.add_argument("--num-batches", type=int, default=0)
# parser.add_argument(
#     "--data-generation", type=str, default="random"
# )  # synthetic or dataset
# parser.add_argument("--data-trace-file", type=str, default="./input/dist_emb_j.log")
# parser.add_argument("--data-set", type=str, default="kaggle")  # or terabyte
# parser.add_argument("--raw-data-file", type=str, default="")
# parser.add_argument("--processed-data-file", type=str, default="")
# parser.add_argument("--data-randomize", type=str, default="total")  # or day or none
# parser.add_argument("--data-trace-enable-padding", type=bool, default=False)
# parser.add_argument("--max-ind-range", type=int, default=-1)
# parser.add_argument("--data-sub-sample-rate", type=float, default=0.0)  # in [0, 1]
# parser.add_argument("--num-indices-per-lookup", type=int, default=10)
# parser.add_argument("--num-indices-per-lookup-fixed", type=bool, default=False)
# parser.add_argument("--num-workers", type=int, default=0)
# parser.add_argument("--memory-map", action="store_true", default=False)
# # training
# parser.add_argument("--mini-batch-size", type=int, default=1)
# parser.add_argument("--nepochs", type=int, default=1)
# parser.add_argument("--learning-rate", type=float, default=0.01)
# parser.add_argument("--print-precision", type=int, default=5)
# parser.add_argument("--numpy-rand-seed", type=int, default=123)
# parser.add_argument("--sync-dense-params", type=bool, default=True)
# # inference
# parser.add_argument("--inference-only", action="store_true", default=False)
# # onnx
# parser.add_argument("--save-onnx", action="store_true", default=False)
# # gpu
# parser.add_argument("--use-gpu", action="store_true", default=False)
# # debugging and profiling
# parser.add_argument("--print-freq", type=int, default=1)
# parser.add_argument("--test-freq", type=int, default=-1)
# parser.add_argument("--test-mini-batch-size", type=int, default=-1)
# parser.add_argument("--test-num-workers", type=int, default=-1)
# parser.add_argument("--print-time", action="store_true", default=False)
# parser.add_argument("--debug-mode", action="store_true", default=False)
# parser.add_argument("--enable-profiling", action="store_true", default=False)
# parser.add_argument("--plot-compute-graph", action="store_true", default=False)
# # store/load model
# parser.add_argument("--save-model", type=str, default="")
# parser.add_argument("--load-model", type=str, default="")
# # mlperf logging (disables other output and stops early)
# parser.add_argument("--mlperf-logging", action="store_true", default=False)
# # stop at target accuracy Kaggle 0.789, Terabyte (sub-sampled=0.875) 0.8107
# parser.add_argument("--mlperf-acc-threshold", type=float, default=0.0)
# # stop at target AUC Terabyte (no subsampling) 0.8025
# parser.add_argument("--mlperf-auc-threshold", type=float, default=0.0)
# parser.add_argument("--mlperf-bin-loader", action="store_true", default=False)
# parser.add_argument("--mlperf-bin-shuffle", action="store_true", default=False)
# # LR policy
# parser.add_argument("--lr-num-warmup-steps", type=int, default=0)
# parser.add_argument("--lr-decay-start-step", type=int, default=0)
# parser.add_argument("--lr-num-decay-steps", type=int, default=0)
# args = parser.parse_args([])

# if args.mlperf_logging:
#     print("command line args: ", json.dumps(vars(args)))

# ### some basic setup ###
# np.random.seed(args.numpy_rand_seed)
# np.set_printoptions(precision=args.print_precision)
# torch.set_printoptions(precision=args.print_precision)
# torch.manual_seed(args.numpy_rand_seed)

# if args.test_mini_batch_size < 0:
#     # if the parameter is not set, use the training batch size
#     args.test_mini_batch_size = args.mini_batch_size
# if args.test_num_workers < 0:
#     # if the parameter is not set, use the same parameter for training
#     args.test_num_workers = args.num_workers

# use_gpu = args.use_gpu and torch.cuda.is_available()
# if use_gpu:
#     torch.cuda.manual_seed_all(args.numpy_rand_seed)
#     torch.backends.cudnn.deterministic = True
#     device = torch.device("cuda", 0)
#     ngpus = torch.cuda.device_count()  # 1
#     print("Using {} GPU(s)...".format(ngpus))
# else:
#     device = torch.device("cpu")
#     print("Using CPU...")

# ### prepare training data ###
# ln_bot = np.fromstring(args.arch_mlp_bot, dtype=int, sep="-")
# # input data
# if args.data_generation == "dataset":

#     train_data, train_ld, test_data, test_ld = dp.make_criteo_data_and_loaders(args)
#     nbatches = args.num_batches if args.num_batches > 0 else len(train_ld)
#     nbatches_test = len(test_ld)

#     ln_emb = train_data.counts
#     # enforce maximum limit on number of vectors per embedding
#     if args.max_ind_range > 0:
#         ln_emb = np.array(
#             list(
#                 map(
#                     lambda x: x if x < args.max_ind_range else args.max_ind_range,
#                     ln_emb,
#                 )
#             )
#         )
#     m_den = train_data.m_den
#     ln_bot[0] = m_den
# else:
#     # input and target at random
#     ln_emb = np.fromstring(args.arch_embedding_size, dtype=int, sep="-")
#     m_den = ln_bot[0]
#     train_data, train_ld = dp.make_random_data_and_loader(args, ln_emb, m_den)
#     nbatches = args.num_batches if args.num_batches > 0 else len(train_ld)

# ### parse command line arguments ###
# m_spa = args.arch_sparse_feature_size
# num_fea = ln_emb.size + 1  # num sparse + num dense features
# m_den_out = ln_bot[ln_bot.size - 1]
# if args.arch_interaction_op == "dot":
#     # approach 1: all
#     # num_int = num_fea * num_fea + m_den_out
#     # approach 2: unique
#     if args.arch_interaction_itself:
#         num_int = (num_fea * (num_fea + 1)) // 2 + m_den_out
#     else:
#         num_int = (num_fea * (num_fea - 1)) // 2 + m_den_out
# elif args.arch_interaction_op == "cat":
#     num_int = num_fea * m_den_out
# else:
#     sys.exit(
#         "ERROR: --arch-interaction-op="
#         + args.arch_interaction_op
#         + " is not supported"
#     )
# arch_mlp_top_adjusted = str(num_int) + "-" + args.arch_mlp_top
# ln_top = np.fromstring(arch_mlp_top_adjusted, dtype=int, sep="-")

# # sanity check: feature sizes and mlp dimensions must match
# if m_den != ln_bot[0]:
#     sys.exit(
#         "ERROR: arch-dense-feature-size "
#         + str(m_den)
#         + " does not match first dim of bottom mlp "
#         + str(ln_bot[0])
#     )
# if args.qr_flag:
#     if args.qr_operation == "concat" and 2 * m_spa != m_den_out:
#         sys.exit(
#             "ERROR: 2 arch-sparse-feature-size "
#             + str(2 * m_spa)
#             + " does not match last dim of bottom mlp "
#             + str(m_den_out)
#             + " (note that the last dim of bottom mlp must be 2x the embedding dim)"
#         )
#     if args.qr_operation != "concat" and m_spa != m_den_out:
#         sys.exit(
#             "ERROR: arch-sparse-feature-size "
#             + str(m_spa)
#             + " does not match last dim of bottom mlp "
#             + str(m_den_out)
#         )
# else:
#     if m_spa != m_den_out:
#         sys.exit(
#             "ERROR: arch-sparse-feature-size "
#             + str(m_spa)
#             + " does not match last dim of bottom mlp "
#             + str(m_den_out)
#         )
# if num_int != ln_top[0]:
#     sys.exit(
#         "ERROR: # of feature interactions "
#         + str(num_int)
#         + " does not match first dimension of top mlp "
#         + str(ln_top[0])
#     )

# # assign mixed dimensions if applicable
# if args.md_flag:
#     m_spa = md_solver(
#         torch.tensor(ln_emb),
#         args.md_temperature,  # alpha
#         d0=m_spa,
#         round_dim=args.md_round_dims,
#     ).tolist()

# # test prints (model arch)
# if args.debug_mode:
#     print("model arch:")
#     print(
#         "mlp top arch "
#         + str(ln_top.size - 1)
#         + " layers, with input to output dimensions:"
#     )
#     print(ln_top)
#     print("# of interactions")
#     print(num_int)
#     print(
#         "mlp bot arch "
#         + str(ln_bot.size - 1)
#         + " layers, with input to output dimensions:"
#     )
#     print(ln_bot)
#     print("# of features (sparse and dense)")
#     print(num_fea)
#     print("dense feature size")
#     print(m_den)
#     print("sparse feature size")
#     print(m_spa)
#     print(
#         "# of embeddings (= # of sparse features) "
#         + str(ln_emb.size)
#         + ", with dimensions "
#         + str(m_spa)
#         + "x:"
#     )
#     print(ln_emb)

#     print("data (inputs and targets):")
#     for j, (X, lS_o, lS_i, T) in enumerate(train_ld):
#         # early exit if nbatches was set by the user and has been exceeded
#         if nbatches > 0 and j >= nbatches:
#             break

#         print("mini-batch: %d" % j)
#         print(X.detach().cpu().numpy())
#         # transform offsets to lengths when printing
#         print(
#             [
#                 np.diff(S_o.detach().cpu().tolist() + list(lS_i[i].shape)).tolist()
#                 for i, S_o in enumerate(lS_o)
#             ]
#         )
#         print([S_i.detach().cpu().tolist() for S_i in lS_i])
#         print(T.detach().cpu().numpy())

# ndevices = min(ngpus, args.mini_batch_size, num_fea - 1) if use_gpu else -1

# ### construct the neural network specified above ###
# # WARNING: to obtain exactly the same initialization for
# # the weights we need to start from the same random seed.
# # np.random.seed(args.numpy_rand_seed)
# dlrm = DLRM_Net(
#     m_spa,
#     ln_emb,
#     ln_bot,
#     ln_top,
#     arch_interaction_op=args.arch_interaction_op,
#     arch_interaction_itself=args.arch_interaction_itself,
#     sigmoid_bot=-1,
#     sigmoid_top=ln_top.size - 2,
#     sync_dense_params=args.sync_dense_params,
#     loss_threshold=args.loss_threshold,
#     ndevices=ndevices,
#     qr_flag=args.qr_flag,
#     qr_operation=args.qr_operation,
#     qr_collisions=args.qr_collisions,
#     qr_threshold=args.qr_threshold,
#     md_flag=args.md_flag,
#     md_threshold=args.md_threshold,
# )
# for j, (X, lS_o, lS_i, T) in enumerate(train_ld):
#     Z = dlrm(X, lS_o, lS_i)
# graph = trace(dlrm, (X, lS_o, lS_i))
# print(graph)

In [4]:
# Just preload the graph for fast experiments 
# Create Graph

# for name, model in models.__dict__.items():
# #     print(name)
#     if not name.islower() or name.startswith("__") or not callable(model):
#         continue
#     if "vgg11" in name and "vgg11_bn" not in name :
#         inputs = torch.randn(1, 3, 299, 299)
#         vgg11_graph = trace(model().eval(), inputs)
#         break
#         print(vgg11_graph)
#     if "resnet50" in name:
# #         model = model().eval()
#         inputs = torch.randn(1, 3, 100, 100)
#         graph = trace(model, inputs)

In [5]:
# from transformers import BertModel, BertConfig
# from transformers import GPT2Model, GPT2Config
# # configuration = BertConfig()
# # model = BertModel(configuration)

# configuration = GPT2Config()
# model = GPT2Model(configuration)
# # model.configMM

# # tokenizer = torch.hub.load('huggingface/pytorch-pretrained-BERT', 'tokenizer', 'bert-base-cased', do_basic_tokenize=False)

# # Tokenized input
# # text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
# # tokenized_text = tokenizer.tokenize(text)
# # indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# # print(indexed_tokens)
# ### Get the hidden states computed by `bertModel`
# # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
# segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
# indexed_tokens = [101, 2627, 1108, 3104, 1124, 15703, 136, 102, 3104, 1124, 15703, 1108, 170, 16797, 8284, 102]

# # Convert inputs to PyTorch tensors
# segments_tensors = torch.tensor([segments_ids])
# tokens_tensor = torch.tensor([indexed_tokens])

# # model = torch.hub.load('huggingface/pytorch-pretrained-BERT', 'model', 'bert-base-cased')
# model.eval()

# model(tokens_tensor)
# graph = trace(model, tokens_tensor)


In [6]:
# from dgl.nn.pytorch import GraphConv
# import dgl
# import numpy as np

# def build_karate_club_graph():
#     # All 78 edges are stored in two numpy arrays. One for source endpoints
#     # while the other for destination endpoints.
#     src = np.array([1, 2, 2, 3, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 10, 10,
#         10, 11, 12, 12, 13, 13, 13, 13, 16, 16, 17, 17, 19, 19, 21, 21,
#         25, 25, 27, 27, 27, 28, 29, 29, 30, 30, 31, 31, 31, 31, 32, 32,
#         32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 33,
#         33, 33, 33, 33, 33, 33, 33, 33, 33, 33])
#     dst = np.array([0, 0, 1, 0, 1, 2, 0, 0, 0, 4, 5, 0, 1, 2, 3, 0, 2, 2, 0, 4,
#         5, 0, 0, 3, 0, 1, 2, 3, 5, 6, 0, 1, 0, 1, 0, 1, 23, 24, 2, 23,
#         24, 2, 23, 26, 1, 8, 0, 24, 25, 28, 2, 8, 14, 15, 18, 20, 22, 23,
#         29, 30, 31, 8, 9, 13, 14, 15, 18, 19, 20, 22, 23, 26, 27, 28, 29, 30,
#         31, 32])
#     # Edges are directional in DGL; Make them bi-directional.
#     u = np.concatenate([src, dst])
#     v = np.concatenate([dst, src])
#     # Construct a DGLGraph
#     return dgl.DGLGraph((u, v))

# class GCN(nn.Module):
#     def __init__(self, in_feats, hidden_size, num_classes):
#         super(GCN, self).__init__()
#         self.conv1 = GraphConv(in_feats, hidden_size)
#         self.conv2 = GraphConv(hidden_size, num_classes)

#     def forward(self, g, inputs):
#         h = self.conv1(g, inputs)
#         h = torch.relu(h)
#         h = self.conv2(g, h)
#         return h

# # The first layer transforms input features of size of 5 to a hidden size of 5.
# # The second layer transforms the hidden layer and produces output features of
# # size 2, corresponding to the two groups of the karate club.
# net = GCN(5, 5, 2)
# # model = net(G,inputs)

# G = build_karate_club_graph()
# embed = nn.Embedding(34, 5)  # 34 nodes with embedding dim equal to 5
# G.ndata['feat'] = embed.weight
# inputs = embed.weight

# graph = trace(net, (G,inputs))


In [1]:
# !pip install -q torch-scatter==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
# !pip install -q torch-sparse==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
# !pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

In [8]:
# !pip install -q torch-scatter==latest+cu92 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
# !pip install -q torch-sparse==latest+cu92 -f https://pytorch-geometric.com/whl/torch-1.6.0.html


In [10]:
# import torch
# from torch.nn import Linear
# from torch_geometric.nn import GCNConv


# class GCN(torch.nn.Module):
#     def __init__(self):
#         super(GCN, self).__init__()
#         torch.manual_seed(12345)
#         self.conv1 = GCNConv(dataset.num_features, 4)
#         self.conv2 = GCNConv(4, 4)
#         self.conv3 = GCNConv(4, 2)
#         self.classifier = Linear(2, dataset.num_classes)

#     def forward(self, x, edge_index):
#         h = self.conv1(x, edge_index)
#         h = h.tanh()
#         h = self.conv2(h, edge_index)
#         h = h.tanh()
#         h = self.conv3(h, edge_index)
#         h = h.tanh()  # Final GNN embedding space.
        
#         # Apply a final (linear) classifier.
#         out = self.classifier(h)

#         return out, h

# model = GCN()
# from torch_geometric.datasets import KarateClub

# dataset = KarateClub()
# data = dataset[0] 
# _, h = model(data.x, data.edge_index)

In [12]:
from generator import Generator
from utils.visualizer import *


In [13]:
import collections
import pdb

import numpy as np
import yaml
import yamlordereddictloader

from utils.logger import create_logger


class Scheduling:
    def __init__(self, hwfile="default.yaml"):
        base_dir = "configs/"
        self.total_cycles = 0
        self.logger = create_logger("logs/stats.txt")
        self.config = self.create_config(
            yaml.load(open(base_dir + hwfile), Loader=yamlordereddictloader.Loader)
        )


In [14]:
def create_config(self, config):

    self.logger.info("Config Statistics : ")

    self.mle = config["memory_levels"]
    self.mem_energy = np.zeros((self.mle))
    self.compute_energy = 0
    self.mem_read_access = np.zeros((self.mle))
    self.mem_write_access = np.zeros((self.mle))
    self.mem_size = np.zeros((self.mle))
    self.mem_util = np.zeros((self.mle))
    self.mem_free = np.zeros((self.mle))
    self.mem_read_bw = np.zeros((self.mle))
    self.mem_write_bw = np.zeros((self.mle))
    self.internal_bandwidth_time = 0
    self.total_cycles = 0 
    self.bandwidth_idle_time = 0
    self.compute_idle_time = 0
    self.mem_size_idle_time = 0
    self.force_connectivity = False
    mm_compute = config["mm_compute"]
    vector_compute = config["vector_compute"]

    if config["mm_compute"]["class"] == "systolic_array":
        config["mm_compute_per_cycle"] = (
            ((mm_compute["size"]) ** 2) * mm_compute["N_PE"] / 2
        )
        config["comp_bw"] = (
            mm_compute["size"] * mm_compute["N_PE"] * mm_compute["frequency"] * 2
        )

        self.logger.info(
            "MM Compute per cycle : %d", config["mm_compute_per_cycle"]
        )
        self.logger.info("Compute Bandwidth Required : %d", config["comp_bw"])

    if config["mm_compute"]["class"] == "mac":
        config["mm_compute_per_cycle"] = (
            ((mm_compute["size"])) * mm_compute["N_PE"] / 2
        )
        config["comp_read_bw"] = (
            mm_compute["size"] * mm_compute["N_PE"] * mm_compute["frequency"] * 2
        )

    for i in range(self.mle):
        memory = config["memory"]["level" + str(i)]
        self.mem_read_bw[i] = (
            memory["frequency"]
            * memory["banks"]
            * memory["read_ports"]
            * memory["width"]
        )
        self.mem_write_bw[i] = (
            memory["frequency"]
            * memory["banks"]
            * memory["write_ports"]
            * memory["width"]
        )
        self.mem_size[i] = memory["size"]

        self.logger.info(
            "Memory at Level %d, Read Bandwidth %d Write Bandwidth %d",
            i,
            self.mem_read_bw[i],
            self.mem_write_bw[i],
        )

    return config

In [15]:
def run(self, graph):

    """
     Check both size, utilization and bandwidths at every node
     What about memory size that can also get exhausted ?
     So if memory size is exhausted, then have to go to a previous level and write there ?
     if any level utilization is exhausted then only the immediate memory required will be kept.
     if the memory is empty in size, but is not bandwidth, it is useless?
     Cannot do prefetching
     Read access of the next node will decrease
     Bandwidth is available but size is not?, can do prefetching, but now the memory fetches have to check, 
     whether to do fetches of the same node or a different node
     Say bandwidth at level0 is sufficient, at level1 is insufficient, then at level1 we have a bottlenecks
     slower so it will take its own time
     Do vector operations in the meantime perhaps ? 

    """

    config = self.config

    read_bw_req = []
    write_bw_req = []
    read_bw_actual = []
    write_bw_actual = []
    cycles = []
    free_cycles = []
    transferable_checkpointed_edge = []
    all_checkpointed_edge = []
    self.mem_util_log=[]
    self.mem_util_full=[]
    # Mem Fetch time of the last Nodes
#     print(self.mem_free[0], self.mem_util[0], self.mem_size[0])

    mem_free = True
    for n, node in enumerate(graph.nodes):

        # These are last level read/write accesses
        compute_expense, read_access, write_access = node.get_stats()
#         print(read_access, node.in_edge_mem)
        
        self.mem_read_access[1]+=(read_access)
      
        self.logger.info(node.get_stats())
        self.mem_util[0] += node.in_edge_mem
        
        # Total Free memory
        for i in range(self.mle - 1):
            self.mem_free[i] = self.mem_size[i] - self.mem_util[i]
            
#         print("2",self.mem_free[0], self.mem_util[0], self.mem_size[0])
        time_compute = compute_expense / config["mm_compute_per_cycle"]
        read_bw_ll = read_access / (time_compute)
        write_bw_ll = write_access / (time_compute)
        step_cycles = time_compute
        read_bw_req.append(read_bw_ll)
        write_bw_req.append(write_bw_ll)
        free_cycles.append(step_cycles)
#         print("bandwidth",read_bw_ll, write_bw_ll, step_cycles) 
        
        if self.mem_free[0] < node.mem_util:
            mem_free = False
            # node mem_util = output edge
            self.logger.info("Memory size is too low/ Memory is Full")
            self.logger.info("Node or Node memory Requirements too high")
            # Rearrange the checkpointed_nodes
            # rearrange = True

            # Is it possible now : Otherwise update the last level memory bandwidth requirements
            if(self.mem_free[0]<0):      
                step_cycles += 2*(node.in_edge_mem//(self.mem_free[0]+node.in_edge_mem) + 1)*(
                   (self.mem_free[0]+node.in_edge_mem)  / self.mem_read_bw[self.mle - 1]
                ) + time_compute*(node.in_edge_mem//(self.mem_free[0]+node.in_edge_mem) + 1)/256
                # Change this later with the number of solid total cycles
                self.mem_size_idle_time += 2*(node.in_edge_mem//self.mem_read_bw[self.mle - 1]) + time_compute/256*(node.in_edge_mem//(self.mem_free[0]+node.in_edge_mem) + 1)
                self.mem_read_access[0]+=node.in_edge_mem*(node.in_edge_mem//(self.mem_free[0]+node.in_edge_mem)) 
                self.mem_write_access[0]+=node.in_edge_mem*(node.in_edge_mem//(self.mem_free[0]+node.in_edge_mem)) 
        else:
            self.mem_util[0] += node.mem_util
            self.mem_free[0] -= node.mem_util
#         print("2.5",self.mem_free[0], self.mem_util[0], self.mem_size[0])
        self.mem_util_log.append(self.mem_util[0])
        self.mem_read_access[0]+=node.mem_util
        self.mem_write_access[0]+=node.mem_util
            
        assert(self.mem_free[0] < self.mem_size[0])
        # Last level memory fetch takes more time, so that may be a bottleneck
        bandwidth_available = read_bw_ll < self.mem_read_bw[self.mle - 1]
        
        # If Bandwidth is not available : Cannot Prefetch
        if (bandwidth_available) == False:
            step_cycles += (
                read_bw_ll / self.mem_read_bw[self.mle - 1]
            - 1) * time_compute
            self.bandwidth_idle_time += (
                read_bw_ll / self.mem_read_bw[self.mle - 1]
            - 1) * time_compute

        # If memory is not free for the next node and Bandwidth is available : Move nodes back and forth
        # if(total_mem_free[0] == 0 and (bandwidth_available)):
        # for(nodes in checkpointed_nodes):
        # checkpointed but not immediate node

        # Check if memory is free and Bandwidth available : From the Data Dependence Graph, Prefetch new node

        # pdb.set_trace()
        if self.mem_free[0] > 0 and (bandwidth_available):
            if n < len(graph.nodes) - 1:
                if self.mem_free[0] > node.next.mem_util:
                    read_access += node.next.mem_util
                    if read_access / step_cycles < self.mem_read_bw[self.mle - 1]:
                        self.mem_util[0] += node.next.mem_util
                        self.mem_write_access[0] += node.next.mem_util
                        self.mem_free[0] -= node.next.mem_util
                        node.next.mem_util = 0
                    else:
                        read_access = self.mem_read_bw[self.mle - 1] * step_cycles
                        self.mem_util[0] += read_access - read_bw_ll * step_cycles
                        self.mem_write_access[0]+=read_access - read_bw_ll * step_cycles
                        self.mem_free[0] -= read_access - read_bw_ll * step_cycles
                        node.next.mem_util = read_access - read_bw_ll * step_cycles

                else:
                    read_access += self.mem_free[0]
                    if read_access / step_cycles < self.mem_read_bw[self.mle - 1]:
                        node.next.mem_util = node.next.mem_util - self.mem_free[0]
                        self.mem_util[0] = self.mem_size[0]
                        self.mem_free[0] = 0
                    else:
                        read_access = self.mem_read_bw[self.mle - 1] * step_cycles
                        self.mem_util[0] += read_access - read_bw_ll * step_cycles
                        self.mem_free[0] -= read_access - read_bw_ll * step_cycles
                        node.next.mem_util = read_access - read_bw_ll * step_cycles
#         print("3",self.mem_free[0], self.mem_util[0], self.mem_size[0])
        self.mem_util_full.append(self.mem_util[0])   
        
            # TODO Consider Write bandwidth for a block read memory or Write Bandwidth  for a endurance purposes
        self.mem_util[0] -= node.in_edge_mem
#         print("4",self.mem_free[0], self.mem_util[0], self.mem_size[0])
    
        if mem_free:
            self.mem_util[0] -= node.mem_util
#         print("5",self.mem_free[0], self.mem_util[0], self.mem_size[0])
        
        self.logger.info(
            "Node operator %r, Step Cycles %d, Read Accesses %d, Write Accesses %d ",
            node.operator,
            step_cycles,
            read_access,
            write_access,
        )
        self.total_cycles += step_cycles
        cycles.append(step_cycles)
        read_bw_actual.append(read_access / step_cycles)
        write_bw_actual.append(write_access / step_cycles)
#         print("actual",read_access / step_cycles, write_access / step_cycles, step_cycles)
#     print("The total cycles are ", self.total_cycles)
    return read_bw_req, write_bw_req, read_bw_actual, write_bw_actual, cycles, free_cycles

In [16]:
Scheduling.create_config = create_config
Scheduling.run = run

In [ ]:
def runner(graph_set):
    """
    Runs the Input Graph
    """
    
    """
    Scheduling works in the following way :
    1. Start with a given/random Hardware point -> Nodes of the graph are scheduled (prefetching)
    
    2. Do the Scheduling with that Point -> Mapping stops here -> Further evaluation is done using accelergy 
    (with values taken from ERT/ART) -> If values not available -> Use plugins for generating these values 
    
    3. Log bottlenecks and work on a different Hardware point -> do this till some realistically
    max, min values are not violated -> Values/Analyses for a different/unavailable point will require full 
    From the technology node set, generate the reference tables using plugins integration of plugins -> 
    Currently using a table at 40nm.   
    
    4. Optimization metric (time/area/energy) of execution in various components, 
    then optimize the metric of execution and take decisions accordingly 
    
    5. Optimize over different workloads ?
    
    """
    
    num_iterations = 10
    generator = Generator()
    bandwidth = [2,10,50,75,100]
    for graph in graph_set:
#     for i in range(len(bandwidth)):
        scheduler = Scheduling()
#         scheduler.mem_read_bw[1] = 10*bandwidth[i]
        read_bw_req, write_bw_req, read_bw_actual, write_bw_actual, cycles, free_cycles = scheduler.run(graph)
        read_bw_limit, write_bw_limit = scheduler.mem_read_bw[scheduler.mle - 1], scheduler.mem_write_bw[scheduler.mle - 1]   
#         bandwidth_bar_graph("read_full.png", read_bw_req, read_bw_actual, read_bw_limit, graph.nodes)
#         cycles_bar_graph("cycles.png", cycles, free_cycles, graph.nodes)
#         mem_util_bar_graph("mem_util.png",scheduler.mem_util_full/scheduler.mem_size[0],scheduler.mem_util_log/scheduler.mem_size[0], graph.nodes)
#         generator.save_statistics(scheduler, True, get_backprop_memory(graph.nodes))
        generator.save_statistics(scheduler)
        
#         for i in range(num_iterations):
#             nexthw = generator.findnext(scheduler)
#             generator.writehwfile(nexthw, str(i) + "hw.yaml")
#             scheduler.create_config(nexthw)
#             read_bw_req, write_bw_req, read_bw_actual, write_bw_actual, cycles, free_cycles = scheduler.run(graph)
#             read_bw_limit, write_bw_limit = scheduler.mem_read_bw[scheduler.mle - 1], scheduler.mem_write_bw[scheduler.mle - 1]   
# #             bandwidth_bar_graph("read_full.png", read_bw_req, read_bw_actual, read_bw_limit, graph.nodes, cycles)
            

In [12]:
# Have to plot memory bandwidth utilization over time

In [13]:
runner([graph])

Time 2.18 2.16876953125 0 0 5 1000000
Energy 5.621051381000001 0.03665625 0.052297421000000004 0.04181651 0.5435481200000001 0.0218 0.0 2.18


In [166]:
# fig = make_dot(model(tokens_tensor, segments_tensors))


In [167]:
# graph

# Technology Tables

In [116]:
import numpy as np
import pandas as pd
d = pd.read_csv('tables/sram.csv')
a = np.array(d)

input = a[:,:3]
# output = a[:,3:]


In [53]:
d = pd.read_csv('plugins/cacti/bus_width.out')


d = d.drop(d.columns[[0,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]], axis=1)

d = d.drop(' Associativity',1)
d = d.drop(' Dynamic search energy (nJ)',1)

In [55]:
a= np.array(d)
np.savetxt('bus.csv',a, fmt='%.18e', delimiter=',')



In [ ]:
bar_graph("read_dummy.png", read_bw_req, read_bw_actual, read_bw_limit, graph.nodes)


In [232]:
mem_table = np.array(pd.read_csv("tables/sram.csv", header=None))
a = mem_table[np.where(mem_table[:, 1] == 4)]
a = a[np.where(a[:, 2] == 32)]

compute:
    name       : MACs
        class      : mac
        attributes :            
        instances       : 256
        meshX           : 16
        word-bits       : 16

    memory: 
        name       : DRAM
        class      : DRAM
        attributes :
        instances       : 1
        word-bits       : 16
  
    name       : OutputBuffer
        class      : SRAM
        attributes :
        entries         : 1024  # 64 * 16 = 1024
        instances       : 1
        meshX           : 1
        word-bits       : 16
        block-size      : 16
        read_bandwidth  : 16 # words/cycle
        write_bandwidth : 16 # words/cycle

        name       : InputBuffer
            class      : SRAM
            attributes :
            entries         : 1024 # 64 * 16 = 1024
            instances       : 1
            meshX           : 1
            word-bits       : 16
            block-size      : 16
            read_bandwidth  : 16 # words/cycle
            write_bandwidth : 16 # words/cycle

        name       : PsumRegFile
            class      : regfile
            attributes :
            entries         : 1
            instances       : 16
            meshX           : 16
            word-bits       : 16
            cluster-size    : 16
            read_bandwidth  : 1  # words/cycle
            write_bandwidth : 1  # words/cycle
            
        name       : WeightBuffer
            class      : regfile
            attributes :
            entries         : 64
            instances       : 256
            meshX           : 16
            word-bits       : 16
            cluster-size    : 256
            read_bandwidth  : 1  # words/cycle
            write_bandwidth : 1  # words/cycle
    noc:

